# Technical Factor - Strong/Weak Relative Momentum (SRMI)

In [1]:
import dai
import pandas as pd

In [2]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [3]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        IF(close < m_lag(close, 9), (close - m_lag(close, 9)) / m_lag(close, 9), IF(close = m_lag(close, 9), 0, (close - m_lag(close, 9)) / close)) AS SRMI,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(SRMI > 0 AND m_lag(SRMI, 1) <= 0, 1, 0) AS TRBY1,
        IF(SRMI < 0 AND m_lag(SRMI, 1) >= 0, 1, 0) AS TRSL1,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(SRMI > 0 AND (SRMI - m_lag(SRMI, 1)) > 0, 1, 0) AS MTBY1,
        IF(SRMI < 0 AND (SRMI - m_lag(SRMI, 1)) < 0, 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(SRMI < -0.08 AND (SRMI - m_lag(SRMI, 1)) > 0, 1, 0) AS RVBY1,
        IF(SRMI > 0.08 AND (SRMI - m_lag(SRMI, 1)) < 0, 1, 0) AS RVSL1,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(SRMI, 1) <= 0.05 AND SRMI > 0.05, 1, 0) AS BKBY1,
        IF(m_lag(SRMI, 1) >= -0.05 AND SRMI < -0.05, 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        SRMI,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [4]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,SRMI,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-01-15,600519.SH,-0.011243,0,0,0,0,0,0,0,0
1,2025-01-16,600519.SH,-0.019403,0,0,0,1,0,0,0,0
2,2025-01-17,600519.SH,0.010139,1,0,1,0,0,0,0,0
3,2025-01-20,600519.SH,0.023461,0,0,1,0,0,0,0,0
4,2025-01-21,600519.SH,0.017471,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
229,2025-12-25,600519.SH,0.012238,0,0,1,0,0,0,0,0
230,2025-12-26,600519.SH,0.008490,0,0,0,0,0,0,0,0
231,2025-12-29,600519.SH,0.002717,0,0,0,0,0,0,0,0
232,2025-12-30,600519.SH,-0.013757,0,1,0,1,0,0,0,0
